In [80]:
import keras
keras.__version__

'2.2.5'

In [81]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [82]:

%cd /gdrive
%cd My\ Drive
%cd CSCE421YaleData 
!ls

/gdrive
/gdrive/My Drive
/gdrive/My Drive/CSCE421YaleData
csce_421_final_subject_class_emotion.h5    emotions		  subjects
csce_421_final_subject_class.h5		   glasses
csce_421_final_subject_class_wo_imgAug.h5  Homework5_YaleDataset


In [0]:
import os, shutil

original_dataset_dir = '/gdrive/My Drive/CSCE421YaleData/Homework5_YaleDataset'
base_dir = '/gdrive/My Drive/CSCE421YaleData/subjects'



In [0]:
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir,'test')




In [91]:
from keras import layers
from keras import models
"""
======== Model One =======
===== overfitting is a big problem =====



model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(30, 30, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(15, activation = 'softmax'))
model.summary()
from keras import optimizers
model.compile(loss='categorical_crossentropy', 
              optimizer = optimizers.RMSprop(lr=1e-4),
              metrics = ['acc'])
"""

"""======Model Two======"""
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

"""add dropout to fight overfitting"""
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(15, activation = 'softmax'))
model.summary()
from keras import optimizers
model.compile(loss='categorical_crossentropy', 
              optimizer = optimizers.RMSprop(lr=1e-4),
              metrics = ['acc'])
                        


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 17, 17, 128)       0         
____

In [92]:
from keras.preprocessing.image import ImageDataGenerator

"""Model Two: add data augmentation to generate more samples"""
train_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

"""Model Two: change batch_size from 20 to 32"""
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size =(150,150), 
    batch_size=32, 
    class_mode='categorical') 

"""Model Two: change batch_size from 20 to 32"""
validation_generator = test_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150), 
    batch_size=32, 
    class_mode='categorical')

for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

Found 75 images belonging to 15 classes.
Found 45 images belonging to 15 classes.
data batch shape: (32, 150, 150, 3)
labels batch shape: (32, 15)


In [95]:

history = model.fit_generator(
    train_generator, 
    steps_per_epoch=50, 
    epochs=100, 
    validation_data=validation_generator, 
    validation_steps=50)

model.save('csce_421_final_subject_class_wo_imgAug.h5')


# model.load_weights('csce_421_final_subject_class_wo_imgAug.h5')


Epoch 1/100
50/50 [==============================] - 11s 221ms/step - loss: 0.5050 - acc: 0.8290 - val_loss: 1.0687 - val_acc: 0.7556
Epoch 2/100
50/50 [==============================] - 11s 218ms/step - loss: 0.4635 - acc: 0.8625 - val_loss: 0.8062 - val_acc: 0.7556
Epoch 3/100
50/50 [==============================] - 11s 217ms/step - loss: 0.4442 - acc: 0.8396 - val_loss: 0.9015 - val_acc: 0.7556
Epoch 4/100
50/50 [==============================] - 10s 209ms/step - loss: 0.4553 - acc: 0.8608 - val_loss: 1.0417 - val_acc: 0.7333
Epoch 5/100
50/50 [==============================] - 11s 220ms/step - loss: 0.4412 - acc: 0.8576 - val_loss: 0.9934 - val_acc: 0.7778
Epoch 6/100
50/50 [==============================] - 11s 219ms/step - loss: 0.4360 - acc: 0.8495 - val_loss: 1.1649 - val_acc: 0.7333
Epoch 7/100
50/50 [==============================] - 10s 208ms/step - loss: 0.3954 - acc: 0.8673 - val_loss: 0.8788 - val_acc: 0.7778
Epoch 8/100
50/50 [==============================] - 11s 221ms

In [96]:
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150,150), 
    batch_size=32, 
    class_mode='categorical')

model.evaluate(test_generator)

Found 45 images belonging to 15 classes.
2/2 [==============================] - 0s 57ms/step


[0.4233826133939955, 0.9111111150847541]